In [ ]:
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null
!pip install ../input/transformers/transformers-master/ > /dev/null

In [ ]:
import os
import sys
import glob
import torch

import transformers
import numpy as np
import pandas as pd
import math

import torch
from transformers import BertModel
import torch, random, os, multiprocessing, glob, numpy as np, pandas as pd
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, 
    WEIGHTS_NAME, CONFIG_NAME
)
from tqdm import tqdm_notebook as tqdm
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

transformers.__version__

In [ ]:
train = pd.read_csv("../input/google-quest-challenge/train.csv",)
test = pd.read_csv("../input/google-quest-challenge/test.csv",)

target_cols = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written', 'answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import string

In [ ]:
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [ ]:
train["question_title"] = train["question_title"].map(stripTagsAndUris)
train["question_body"] = train["question_body"].map(stripTagsAndUris)
train["answer"] = train["answer"].map(stripTagsAndUris)

test["question_title"] = test["question_title"].map(stripTagsAndUris)
test["question_body"] = test["question_body"].map(stripTagsAndUris)
test["answer"] = test["answer"].map(stripTagsAndUris)

In [ ]:
def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [ ]:
train["question_title"] = train["question_title"].map(removePunctuation)
train["question_body"] = train["question_body"].map(removePunctuation)
train["answer"] = train["answer"].map(removePunctuation)

test["question_title"] = test["question_title"].map(removePunctuation)
test["question_body"] = test["question_body"].map(removePunctuation)
test["answer"] = test["answer"].map(removePunctuation)

In [ ]:
stops = set(stopwords.words("english"))
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

In [ ]:
train["question_title"] = train["question_title"].map(removeStopwords)
train["question_body"] = train["question_body"].map(removeStopwords)
train["answer"] = train["answer"].map(removeStopwords)

test["question_title"] = test["question_title"].map(removeStopwords)
test["question_body"] = test["question_body"].map(removeStopwords)
test["answer"] = test["answer"].map(removeStopwords)

In [ ]:
train = clean_data(train, ['answer', 'question_body', 'question_title'])
test = clean_data(test, ['answer', 'question_body', 'question_title'])

In [ ]:
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512, t_max_len=30, q_max_len=120, a_max_len=358):
    
    #293+239+30 = 508 + 4 = 512
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        t, q, a = _trim_input(t, q, a, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [ ]:
tokenizer = BertTokenizer.from_pretrained("../input/pretrained-bert-models-for-pytorch/bert-base-uncased-vocab.txt")
input_categories = list(train.columns[[1,2,5]]); input_categories

In [ ]:
%%time
outputs = compute_output_arrays(train, columns = target_cols)
inputs = compute_input_arays(train, input_categories, tokenizer, max_sequence_length=512)
test_inputs = compute_input_arays(test, input_categories, tokenizer, max_sequence_length=512)

In [ ]:
inputs[0].shape

In [ ]:
outputs[0].shape

In [ ]:
test_inputs[0].shape

In [ ]:
%%time
lengths = np.argmax(inputs[0] == 0, axis=1)
lengths[lengths == 0] = inputs[0].shape[1]
y_train_torch = torch.tensor(train[target_cols].values, dtype=torch.float32)

In [ ]:
lengths

In [ ]:
min(lengths), max(lengths)

In [ ]:
inputs[1].shape, inputs[2].shape

In [ ]:
inputs[0] = torch.tensor(inputs[0]).numpy()
inputs[1] = torch.tensor(inputs[1]).numpy()
inputs[2] = torch.tensor(inputs[2]).numpy()

lengths = torch.tensor(lengths).numpy()
y_train_torch = y_train_torch.numpy()

In [ ]:
from sklearn.model_selection import train_test_split
X_tr_inputs_ids, X_val_inputs_ids, X_tr_masks, X_val_masks, X_tr_inputs_segs, X_val_inputs_segs, y_train, y_val, X_tr_lengths, X_val_lengths = train_test_split(inputs[0], inputs[1], inputs[2], y_train_torch,lengths,
                                                                              test_size=0.30, random_state=46)

In [ ]:
X_tr_inputs_ids = torch.tensor(X_tr_inputs_ids)
X_val_inputs_ids = torch.tensor(X_val_inputs_ids)

X_tr_masks = torch.tensor(X_tr_masks, dtype=torch.long)
X_val_masks = torch.tensor(X_val_masks, dtype=torch.long)

X_tr_inputs_segs = torch.tensor(X_tr_inputs_segs)
X_val_inputs_segs = torch.tensor(X_val_inputs_segs)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_tr_lengths = torch.tensor(X_tr_lengths)
X_val_lengths = torch.tensor(X_val_lengths)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
import math
from torch.nn import BCEWithLogitsLoss
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from tqdm.notebook import tqdm
from tqdm import trange
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Select a batch size for training
batch_size = 8

train_data = TensorDataset(X_tr_inputs_ids, X_tr_masks, X_tr_inputs_segs, y_train, X_tr_lengths)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batch_size)

validation_data = TensorDataset(X_val_inputs_ids, X_val_masks, X_val_inputs_segs, y_val, X_val_lengths)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batch_size)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
import math
from torch.nn import BCEWithLogitsLoss
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from tqdm.notebook import tqdm
from tqdm import trange
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import logging
import math
import os

import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from transformers import BertConfig, BertPreTrainedModel
bert_model_config = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
config = BertConfig.from_json_file(bert_model_config)
config.num_labels = 30
config

In [ ]:
class BertforSequenceClassification(BertPreTrainedModel):
   
    def __init__(self, config):
        super(BertforSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.pre_classifier = nn.Linear(config.hidden_size, config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = nn.Dropout(0.2)
        self.loss_fct = BCEWithLogitsLoss()

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        bert_output = self.bert(
            input_ids=input_ids, attention_mask=attention_mask, head_mask=head_mask, inputs_embeds=inputs_embeds
        )
        hidden_state = bert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.SELU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        
        if labels is not None:
            
            loss = self.loss_fct(logits.view(-1, self.num_labels),\
                      labels.view(-1, self.num_labels))
            return loss
        else:
            return logits

#         outputs = (logits,) + bert_output[1:]
#         if labels is not None:
#             if self.num_labels == 1:
#                 loss_fct = nn.MSELoss()
#                 loss = loss_fct(logits.view(-1), labels.view(-1))
#             else:
#                 loss_fct = nn.BCEWithLogitsLoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
#             outputs = (loss,) + outputs

#         return outputs  # (loss), logits, (hidden_states), (attentions)
    
model_path = "../input/pretrained-bert-models-for-pytorch/bert-base-uncased/"
model_ex_b = BertforSequenceClassification.from_pretrained(model_path, config=config)

In [ ]:
from transformers import BertConfig

class BertSequenceClassification(torch.nn.Module):
    def __init__(self, num_labels=2):
        super(BertSequenceClassification, self).__init__()
        self.num_labels = num_labels
        bert_model_config = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
        bert_config = BertConfig.from_json_file(bert_model_config)
        bert_config.num_labels = 30
        self.bert = BertModel.from_pretrained('../input/pretrained-bert-models-for-pytorch/bert-base-uncased/', config=bert_config)
        self.dropout = torch.nn.Dropout(0.25)
        self.classifier = torch.nn.Linear(768, num_labels)
        self.loss_fct = BCEWithLogitsLoss()
        
        torch.nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None):
        # last hidden layer
        last_hidden_state = self.bert(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
        # pool the outputs into a mean vector
        mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
        mean_last_hidden_state = self.dropout(mean_last_hidden_state)
        logits = self.classifier(mean_last_hidden_state)
        
        if labels is not None:
            loss = self.loss_fct(logits.view(-1, self.num_labels),\
                      labels.view(-1, self.num_labels))
            return loss
        else:
            return logits
    
    def freeze_xlnet_decoder(self):
        for param in self.xlnet.parameters():
            param.requires_grad = False
    
    def unfreeze_xlnet_decoder(self):
        for param in self.xlnet.parameters():
            param.requires_grad = True
    
    def pool_hidden_state(self, last_hidden_state):
        last_hidden_state = last_hidden_state[0]
        mean_last_hidden_state = torch.mean(last_hidden_state, 1)
        return mean_last_hidden_state

# len(Y_train[0]) = 6
model = BertSequenceClassification(num_labels=len(y_train[0]))
# model.freeze_xlnet_decoder()        

In [ ]:
model.classifier, model_ex_b.classifier

In [ ]:
from scipy.stats import spearmanr

def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
    """
    Train the model and save the model with the lowest validation loss
    """
    crit_function = nn.BCEWithLogitsLoss()
    model.to(device)

    # trange is a tqdm wrapper around the normal python range
    for i in trange(num_epochs, desc="Epoch"):
        # if continue training from saved model
        actual_epoch = start_epoch + i

        # Training

        # Set our model to training mode (as opposed to evaluation mode)
        model.train()

        # Tracking variables
        tr_loss = 0
        num_train_samples = 0

        t = tqdm(total=len(train_data), desc="Training: ", position=0)
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_input_segs, b_labels, b_lengths = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            loss = model(b_input_ids, attention_mask=b_input_mask, token_type_ids = b_input_segs, labels=b_labels)
            # store train loss
            tr_loss += loss.item()
            num_train_samples += b_labels.size(0)
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            #scheduler.step()
            t.update(n=b_input_ids.shape[0])
        t.close()
        # Update tracking variables
        epoch_train_loss = tr_loss/num_train_samples
        train_loss_set.append(epoch_train_loss)

        print("Train loss: {}".format(epoch_train_loss))

        # Validation

        # Put model in evaluation mode to evaluate loss on the validation set
        model.eval()

        # Tracking variables 
        eval_loss = 0
        num_eval_samples = 0

        v_preds = []
        v_labels = []

        # Evaluate data for one epoch
        t = tqdm(total=len(validation_data), desc="Validating: ", position=0)
        for batch in valid_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_input_segs, b_labels, b_lengths = batch
            # Telling the model not to compute or store gradients,
            # saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate validation loss
                preds = model(b_input_ids, attention_mask=b_input_mask, token_type_ids = b_input_segs)
                loss = crit_function(preds, b_labels)
                v_labels.append(b_labels.cpu().numpy())
                v_preds.append(preds.cpu().numpy())
                # store valid loss
                eval_loss += loss.item()
                num_eval_samples += b_labels.size(0)
            t.update(n=b_labels.shape[0])
        t.close()

        v_labels = np.vstack(v_labels)
        v_preds = np.vstack(v_preds)
        print(v_labels.shape)
        print(v_preds.shape)
        rho_val = np.mean([spearmanr(v_labels[:, ind] + np.random.normal(0, 1e-7, v_preds.shape[0]),
                                            v_preds[:, ind] + np.random.normal(0, 1e-7, v_preds.shape[0])).correlation for ind in range(v_preds.shape[1])]
                                )
        epoch_eval_loss = eval_loss/num_eval_samples
        valid_loss_set.append(epoch_eval_loss)

        print("Epoch #{}, training BCE loss: {}, validation BCE loss: ~{}, validation spearmanr: {}"\
                .format(0, epoch_train_loss, epoch_eval_loss, rho_val))

        if lowest_eval_loss == None:
            lowest_eval_loss = epoch_eval_loss
            # save model
        #   save_model(model, model_save_path, actual_epoch,\
        #              lowest_eval_loss, train_loss_set, valid_loss_set)
        else:
            if epoch_eval_loss < lowest_eval_loss:
                lowest_eval_loss = epoch_eval_loss
            # save model
            # save_model(model, model_save_path, actual_epoch,\
            #            lowest_eval_loss, train_loss_set, valid_loss_set)
        print("\n")

    return model, train_loss_set, valid_loss_set


In [ ]:
optimizer = AdamW(model_ex_b.parameters(), lr=2e-5, weight_decay=0.1, correct_bias=False)
model, train_loss_set, valid_loss_set = train(model=model_ex_b,\
                                              num_epochs = 7,
                                              optimizer = optimizer,
                                              train_dataloader = train_dataloader,
                                              valid_dataloader = validation_dataloader,
                                              device='cuda'
                                              )

In [ ]:
test_inputs

In [ ]:
%%time
sequences = np.array(test_inputs[0])
lengths = np.argmax(sequences == 0, axis=1)
lengths[lengths == 0] = sequences.shape[1]

In [ ]:
lengths

In [ ]:
dataset = data.TensorDataset(torch.tensor(test_inputs[0]),
                             torch.tensor(test_inputs[1], dtype=torch.long),
                             torch.tensor(test_inputs[2]),
                            )

test_dataloader = data.DataLoader(dataset, 
                               batch_size=8,  
                               shuffle=False, 
                               drop_last=False
                             )

In [ ]:
next(iter(test_dataloader))

In [ ]:
for items in test_dataloader:
    print(items)

In [ ]:
def generate_predictions(model, dataloader, num_labels, device="cpu", batch_size=8):

    pred_probs = np.array([]).reshape(0, num_labels)

    model.to(device)
    model.eval()

    for X, masks, segments in dataloader:
        X = X.to(device)
        masks = masks.to(device)
        segments = segments.to(device)
        with torch.no_grad():
            logits = model(input_ids=X, attention_mask=masks, token_type_ids=segments)
            logits = logits.sigmoid().detach().cpu().numpy()
            pred_probs = np.vstack([pred_probs, logits])
    return pred_probs
num_labels = len(target_cols)
pred_probs = generate_predictions(model_ex_b, test_dataloader, num_labels=30, device="cuda", batch_size=8)

In [ ]:
df_submit = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
df_submit[target_cols] = pred_probs

In [ ]:
df_submit.to_csv("submission.csv", index = False)
df_submit

In [ ]:
ls